In [1]:
import os
import pathlib

import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models


2023-05-27 19:48:31.176423: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 19:48:32.491202: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# carregando a seed do dataset e definindo a sua rota
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


DATASET_PATH = '/home/rafaelrosendo/figas/ptBR_multi_v01'

SAMPLING_FREQ = 24000

#verificando o uso da gpu
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-05-27 19:48:53.275547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-27 19:48:53.904826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-27 19:48:53.905135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels


In [4]:
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

In [5]:
def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)


In [6]:
if __name__ == "__main__":

	# check data dir
	data_dir = pathlib.Path(DATASET_PATH)
	assert data_dir.exists(), f'{DATASET_PATH} not found.'
	# list speakers
	speakers = np.array(tf.io.gfile.listdir(str(data_dir)))
	print(f'\nSpeakers: {speakers}\n')
	print(f'Total of {len(speakers)} speakers\n')


Speakers: ['spk78' 'spk19' 'spk84' 'spk31' 'spk42' 'spk44' 'spk10' 'spk38' 'spk17'
 'spk66' 'spk0' 'spk33' 'spk26' 'spk23' 'spk13' 'spk48' 'spk65' 'spk57'
 'spk46' 'spk20' 'spk32' 'spk74' 'spk18' 'spk54' 'spk41' 'spk40' 'spk1'
 'spk12' 'spk11' 'spk62' 'spk70' 'spk3' 'spk36' 'spk63' 'spk49' 'spk47'
 'spk43' 'spk5' 'spk59' 'spk8' 'spk24' 'spk83' 'spk14' 'spk4' 'spk45'
 'spk35' 'spk69' 'spk15' 'spk55' 'spk58' 'spk72' 'spk85' 'spk68' 'spk86'
 'spk50' 'spk64' 'spk2' 'spk51' 'spk6' 'spk29' 'spk37' 'spk21' 'spk9'
 'spk56' 'spk77' 'spk34' 'spk22' 'spk27' 'spk16' 'spk82' 'spk61' 'spk39'
 'spk60' 'spk71' 'spk67' 'spk7' 'spk76' 'spk73' 'spk53' 'spk30' 'spk28'
 'spk52' 'spk25']

Total of 83 speakers



In [7]:
# create dataset   
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
		directory=data_dir,
		batch_size=16,
		validation_split=0.2,
		seed=0,
		output_sequence_length=SAMPLING_FREQ,
		subset='both')



Found 12512 files belonging to 83 classes.
Using 10010 files for training.
Using 2502 files for validation.


2023-05-27 19:49:02.148654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-27 19:49:02.148961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-27 19:49:02.149192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
# list classes: speaker names
label_names = np.array(train_ds.class_names)
print(f'\nLabel names: {label_names}\n')

# print format of the data
print(f'Specs of the batches: {train_ds.element_spec}\n')

# removes one dimension from audio
train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

# split the validation data into test and validation
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)


Label names: ['spk0' 'spk1' 'spk10' 'spk11' 'spk12' 'spk13' 'spk14' 'spk15' 'spk16'
 'spk17' 'spk18' 'spk19' 'spk2' 'spk20' 'spk21' 'spk22' 'spk23' 'spk24'
 'spk25' 'spk26' 'spk27' 'spk28' 'spk29' 'spk3' 'spk30' 'spk31' 'spk32'
 'spk33' 'spk34' 'spk35' 'spk36' 'spk37' 'spk38' 'spk39' 'spk4' 'spk40'
 'spk41' 'spk42' 'spk43' 'spk44' 'spk45' 'spk46' 'spk47' 'spk48' 'spk49'
 'spk5' 'spk50' 'spk51' 'spk52' 'spk53' 'spk54' 'spk55' 'spk56' 'spk57'
 'spk58' 'spk59' 'spk6' 'spk60' 'spk61' 'spk62' 'spk63' 'spk64' 'spk65'
 'spk66' 'spk67' 'spk68' 'spk69' 'spk7' 'spk70' 'spk71' 'spk72' 'spk73'
 'spk74' 'spk76' 'spk77' 'spk78' 'spk8' 'spk82' 'spk83' 'spk84' 'spk85'
 'spk86' 'spk9']

Specs of the batches: (TensorSpec(shape=(None, 24000, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))



In [9]:
# check the format of the audio and labels
for example_audio, example_labels in train_ds.take(1):
    print(example_audio.shape)
    print(example_labels.shape)

# extract spectrograms from speech
train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)


2023-05-27 19:49:04.593987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [10010]
	 [[{{node Placeholder/_4}}]]
2023-05-27 19:49:04.594255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [10010]
	 [[{{node Placeholder/_4}}]]


(16, 24000)
(16,)


In [10]:

# look at the spectrogram batches
for example_spectrograms, example_spect_labels in train_spectrogram_ds.take(1):
	break


2023-05-27 19:49:05.039063: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [10010]
	 [[{{node Placeholder/_4}}]]
2023-05-27 19:49:05.039324: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10010]
	 [[{{node Placeholder/_0}}]]


In [11]:
# prefecth batches
train_spectrogram_ds = train_spectrogram_ds.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = val_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = test_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)

In [12]:
# get the input shape (to input at the network)
input_shape = example_spectrograms.shape[1:]
print(f'\nInput shape: {input_shape}\n')
num_labels = len(label_names)


Input shape: (186, 129, 1)



In [13]:
# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
	# Fit the state of the layer to the spectrograms
	# with `Normalization.adapt`.
norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

2023-05-27 19:49:05.250911: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [10010]
	 [[{{node Placeholder/_4}}]]
2023-05-27 19:49:05.251222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [10010]
	 [[{{node Placeholder/_4}}]]


In [77]:
# construção do modelo
model = models.Sequential([
	layers.Input(shape=input_shape),
	# fazendo a arquitetura
	layers.Resizing(32, 32),
		# Normalize.
	norm_layer,
	layers.Conv2D(256, 3, activation='relu'),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
	layers.MaxPooling2D(),
	layers.Dropout(0.5),
	layers.Flatten(),
	layers.Dense(1024, activation='relu'),
	layers.Dropout(0.5),
	layers.Dense(num_labels),
])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_16 (Resizing)      (None, 32, 32, 1)         0         
                                                                 
 normalization (Normalizatio  (None, 32, 32, 1)        3         
 n)                                                              
                                                                 
 conv2d_47 (Conv2D)          (None, 30, 30, 256)       2560      
                                                                 
 conv2d_48 (Conv2D)          (None, 28, 28, 128)       295040    
                                                                 
 conv2d_49 (Conv2D)          (None, 26, 26, 64)        73792     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                

In [78]:
# compilando o modelo
model.compile(
	optimizer=tf.keras.optimizers.Adam(),
	loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
	metrics=['accuracy'],
)


In [79]:

os.environ['CUDA_HOME'] = '/home/rafaelrosendo/anaconda3/pkgs/cudatoolkit-11.8.0-h6a678d5_0'

# treino
EPOCHS = 25
history = model.fit(
	train_spectrogram_ds,
	validation_data=val_spectrogram_ds,
	epochs=EPOCHS,
	#callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Epoch 1/25


2023-05-28 14:13:37.469976: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_16/dropout_32/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


626/626 [==============================] - 6s 8ms/step - loss: 2.2130 - accuracy: 0.5109 - val_loss: 1.5981 - val_accuracy: 0.6522
Epoch 2/25
626/626 [==============================] - 5s 8ms/step - loss: 1.4991 - accuracy: 0.6585 - val_loss: 1.3069 - val_accuracy: 0.6923
Epoch 3/25
626/626 [==============================] - 5s 7ms/step - loss: 1.2496 - accuracy: 0.6971 - val_loss: 1.1948 - val_accuracy: 0.6995
Epoch 4/25
626/626 [==============================] - 5s 7ms/step - loss: 1.0856 - accuracy: 0.7255 - val_loss: 1.1004 - val_accuracy: 0.7252
Epoch 5/25
626/626 [==============================] - 5s 8ms/step - loss: 0.9776 - accuracy: 0.7450 - val_loss: 1.0006 - val_accuracy: 0.7548
Epoch 6/25
626/626 [==============================] - 5s 8ms/step - loss: 0.8907 - accuracy: 0.7655 - val_loss: 0.9953 - val_accuracy: 0.7404
Epoch 7/25
626/626 [==============================] - 5s 8ms/step - loss: 0.8118 - accuracy: 0.7847 - val_loss: 0.8884 - val_accuracy: 0.7740
Epoch 8/25
626/62

In [80]:

	# acurácia e perda
	model.evaluate(test_spectrogram_ds, return_dict=True)

79/79 [==============================] - 0s 2ms/step - loss: 0.8720 - accuracy: 0.7847


{'loss': 0.8719855546951294, 'accuracy': 0.7846890091896057}